# Manual Curation Importing Back To Full CSV

## THINGS THAT NEED TO BE CHANGED BEFORE RUNNING CODE
Takes contested dataframe and replaces rows with the correct entry winners

In [43]:
#File path of the full csv
# Ex. r"C:\Users\albert\Desktop\20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.csv"
full_csv_file_path = r"./proc/rce_pilot_2_nose_pokes.pkl"

#file path of the contested csv
# Ex. r"C:\Users\albert\Desktop\20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1contested_frames.csv"
contested_csv_file_path = r"C:\Users\yongc\Desktop\Leo Code\20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1contested_frames.csv"

# Where you want your file to be saved
# Ex. r"C:\Users\yongc\Destop\\"
full_csv_combined_save_file_path = r"C:\Users\yongc\Desktop\\"
full_csv_combined_save_name = r"20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1_corrected.csv"


In [44]:
import os
import glob
import git
import sys
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

In [45]:
nose_poke_full_df = pd.read_pickle(full_csv_file_path)
manual_df = pd.read_csv(contested_csv_file_path)

In [46]:
nose_poke_full_df.head()

,session_dir,tone_frames,box_1_port_entry_frames,video_name,session_path,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,...,nose_index,entry_length,box_1_port_entry_start,box_1_port_entry_end,sliced_locations,nose_coordinates,nose_to_reward_port_distance,nose_to_reward_port_distance_x,entry_winner,contested
0,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[192, 248]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[192745, 249350]",...,2,56,192,248,"{'1.1': [[[47.88 11.6 ], [46.72 12.984], [48....","{'1.1': [[48.3, 13.7], [47.84, 13.695], [47.75...","{'1.1': 22.351771133170907, '1.2': 0.988053123...","{'1.1': 3.033430232558146, '1.2': 0.9709302325...",1.2,False
1,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[389, 405]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[389747, 407142]",...,2,16,389,405,"{'1.1': [[[46.1 33.44], [43.97 34.94], [46.16...","{'1.1': [[46.16, 36.0], [46.12, 35.97], [46.16...","{'1.1': 0.8902350171923165, '1.2': 25.90464148...","{'1.1': 0.8771802325581461, '1.2': 6.564680232...",1.1,False
2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[916, 929]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[917544, 931741]",...,2,13,916,929,"{'1.1': [[[61.34 18. ], [58.6 18.38], [60.25...","{'1.1': [[60.25, 19.8], [60.78, 19.5], [60.78,...","{'1.1': 21.949273716111037, '1.2': 1.066196743...","{'1.1': 14.970930232558146, '1.2': 0.908430232...",1.2,False
3,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[929, 948]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[932141, 950539]",...,2,19,929,948,"{'1.1': [[[59.7 15.125], [60.84 16.25], [61....","{'1.1': [[61.47, 16.03], [61.22, 15.4], [61.0,...","{'1.1': 25.589320821862895, '1.2': 0.630309449...","{'1.1': 16.189680232558146, '1.2': 0.345930232...",1.2,False
4,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[961, 983]",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",7977066,76318450,"[[1126742, 1326741], [3526740, 3726740], [5826...","[963342, 986354]",...,2,22,961,983,"{'1.1': [[[58.62 12.96], [60.28 14.03], [59.75...","{'1.1': [[59.75, 13.06], [59.88, 14.26], [59.9...","{'1.1': 26.99243314043086, '1.2': 0.6916250894...","{'1.1': 14.470930232558146, '1.2': 0.408430232...",1.2,False


In [47]:
for i in range(len(manual_df.columns)):
    nose_poke_full_df.loc[(manual_df['Unnamed: 0'][i]),'entry_winner'] = manual_df['entry_winner'][i]

# Make sure that whatever you changed is correct

In [51]:
filtered_df = nose_poke_full_df[nose_poke_full_df['contested'] == True]
filtered_df[['box_1_port_entry_frames', 'entry_winner', 'contested']]

,box_1_port_entry_frames,entry_winner,contested
155,"[9720, 9868]",1.2,True
156,"[9870, 9899]",1.2,True
165,"[10516, 10518]",1.2,True
167,"[10640, 10723]",1.1,True
211,"[14368, 14606]",1.2,True
266,"[17958, 17975]",1.2,True
267,"[17977, 17980]",1.2,True
268,"[17987, 17988]",1.2,True
341,"[21890, 21941]",1.1,True
379,"[23424, 23441]",1.1,True


In [49]:
nose_poke_full_df.to_csv(full_csv_combined_save_file_path+full_csv_combined_save_name)